In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from tqdm import tqdm_notebook

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df = pd.read_csv('Final Indemed Sheet.csv')

In [3]:
ibl = pd.read_csv('K:\Internship\DME\Parent Extraction\ItemBrandList.csv') 

In [4]:
pd.set_option('display.max_columns',None)

In [5]:
df.head()

,Item Name/Number,Item Options,SIZE,COLOR,ITEM OPTION,PACKAGING,QUANTITY,LENGTH,SCENT,DIMENSION,MATERIAL,SHAPE,STERILITY,CHOICE,FRENCH GAUGE,VOLUME,ABSORBANCY,WEIGHT,Indemed Item,Matrix Type,Subitem of,Display Name/Code,Vendor Name/Code,Manufacturer,UOM to Each,UOM to Each Type,Units Type,Stock Description,Purchase Price,Base Price,Item Defined Cost,Shipping Category,Shipping Rate,Logo - Free Shipping,Actual Shipping,Application,Product Name,Detailed Description,Web Store Description,Page Title,Meta Tag HTML,Warranty,Return Policy,Attribute Set,ItemImageURL,ItemDescription,Other Options,ITEM LENGTH,USER TYPE,PAD SIZE,ABSORBENCY
0,3M1882-MI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parent Matrix Item,NaN,1882,Indemed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tegaderm I.V. Advanced Securement Dressing 2-3...,"<p>Tegaderm I.V. Advanced Securement Dressing,...","<p>Tegaderm I.V. Advanced Securement Dressing,...","Tegaderm I.V. Advanced Securement Dressing, 2-...","<meta name= ""description"" content=""Buy Tegader...",NaN,NaN,Supply Compression,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3M1882-EA,SIZE|STERILITY,"2.75"" x 3.375""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sterile,NaN,NaN,NaN,NaN,NaN,881882,Child Matrix Item,NaN,1882,Indemed,3M,1.0,EA,Each,1/Each,1.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Tegaderm I.V. Advanced Securement Dressing, 2-...",NaN,NaN,NaN,NaN,NaN,NaN,Supply Compression,...,"Tegaderm I.V. Advanced Securement Dressing, 2....",NaN,NaN,NaN,NaN,NaN
2,3M1882-BX,SIZE|STERILITY,"2.75"" x 3.375""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sterile,NaN,NaN,NaN,NaN,NaN,881882,Child Matrix Item,NaN,1882,Indemed,3M,50.0,BX,Box,50/Box,60.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Tegaderm I.V. Advanced Securement Dressing, 2-...",NaN,NaN,NaN,NaN,NaN,NaN,Supply Compression,...,"Tegaderm I.V. Advanced Securement Dressing, 2....",NaN,NaN,NaN,NaN,NaN
3,3M2013-MI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parent Matrix Item,NaN,2013,Indemed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Adaptic Non-adhering Dressing 3"" x 8""","<p>Adaptic Non-Adhering Dressing 3"" x 8"" Steri...","<p>Adaptic Non-Adhering Dressing 3"" x 8"" Steri...","Adaptic Non-Adhering Dressing 3"" x 8"" Sterile ...","<meta name= ""description"" content=""Buy Adaptic...",NaN,NaN,Supply Compression,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3M2013-CA,SIZE,"3"" x 8""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,532013,Child Matrix Item,NaN,2013,Indemed,3M,216.0,CA,Case,216/Case,426.71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Adaptic Non-Adhering Dressing 3"" x 8"" Sterile 3's",NaN,NaN,NaN,NaN,NaN,NaN,Supply Compression,532013B.jpg ...,"Adaptic Non-Adhering Dressing 3"" x 8"" Sterile 3's",NaN,NaN,NaN,NaN,NaN


In [6]:
for index,rows in tqdm_notebook(df.iterrows(), total=df.shape[0]):
    if 'EA' in rows['Item Name/Number']:
        df.loc[index,'Item Name/Number'] = rows['Item Name/Number'][:-3]
    if pd.isnull(rows['SIZE'])!=True:
        if '"' in rows['SIZE'] or '""' in rows['SIZE']:
            df.loc[index,'SIZE'] = rows['SIZE'].replace('"',' Inch')            

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


In [7]:
mf_list = pd.Series(ibl['Name'],name = 'Name')

In [8]:
mf_list = mf_list.unique()

In [9]:
df_list = df['Manufacturer'].unique()

In [10]:
d = {}
for i in df_list:
    max_ratio = 0
    if pd.isnull(i):
        continue
    for j in mf_list:
        ratio = fuzz.partial_ratio(i.lower(),j.lower())
        if ratio > max_ratio:
            max_ratio = ratio
            d[i] = j

In [13]:
for index,rows in df.iterrows():
    key = rows['Manufacturer']
    if pd.isnull(key):
        continue
    else:
        df.loc[index,'Manufacturer'] = d[key]

In [14]:
df.to_csv('Netsuite Final Feed Indemed.csv',index = False)